In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
import os
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/Google_clustering')
pd.set_option('display.max_rows', 500)


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from datetime import datetime as dt


In [4]:
!ls

 Clustering_data.csv	  My_location_history.png
'Location History.json'   Visualizing_location_history


In [5]:
df_gps = pd.read_json('Location History.json')


In [6]:
print('There are {:,} rows'.format(len(df_gps)))

There are 293,536 rows


In [7]:
df_gps["locations"].head()

0    {'timestampMs': '1465965720372', 'latitudeE7':...
1    {'timestampMs': '1465965739870', 'latitudeE7':...
2    {'timestampMs': '1468842511651', 'latitudeE7':...
3    {'timestampMs': '1468842535413', 'latitudeE7':...
4    {'timestampMs': '1468842550427', 'latitudeE7':...
Name: locations, dtype: object

In [8]:
df_gps['lat'] = df_gps['locations'].map(lambda x: x['latitudeE7']) / 10.**7
df_gps['lon'] = df_gps['locations'].map(lambda x: x['longitudeE7']) / 10.**7

In [9]:
df_gps['timestamp_s'] = (df_gps['locations'].map(lambda x: x['timestampMs']).astype(float) / 1000).astype(int)
df_gps['datetime'] = pd.to_datetime(df_gps['timestamp_s'], unit='s')

In [10]:

# filter out points with altitudes above 3000 meters to remove airplane data
df_gps['altitude'] = df_gps['locations'].map(lambda x: x['altitude'] if 'altitude' in x else None)
mask = (df_gps['altitude'] < 3000) | (pd.isnull(df_gps['altitude']))
df_gps = df_gps[mask]

In [11]:
kms_per_radian = 6371.0088

In [12]:
df_gps_sample=df_gps.head(90000)

In [13]:
# coords = df_gps.as_matrix(columns=["lat", "lon"])   
coords = df_gps_sample[["lat", "lon"]].to_numpy()

In [14]:
coords.shape

(90000, 2)

In [ ]:
db = DBSCAN(eps=0.05/6371.0088, min_samples=2, algorithm='ball_tree', metric='haversine').fit_predict(np.radians(coords))
df_gps_sample['cluster'] = db

In [16]:
df_gps_sample['cluster'].nunique()

629

In [17]:
df_gps_sample[df_gps_sample['cluster']==708]

,locations,lat,lon,timestamp_s,datetime,altitude,cluster


In [18]:
clusters=df_gps_sample.groupby(["cluster"]).lat.count().sort_values(ascending=False)

In [19]:
clusters.head(5)

cluster
 1      64397
 26      9889
 22      3864
 228     2307
-1        573
Name: lat, dtype: int64

In [20]:
df_gps_sample[df_gps_sample["cluster"]==91].head(20)

,locations,lat,lon,timestamp_s,datetime,altitude,cluster
2383,"{'timestampMs': '1483119678202', 'latitudeE7':...",10.760552,79.839303,1483119678,2016-12-30 17:41:18,NaN,91
2384,"{'timestampMs': '1483119699222', 'latitudeE7':...",10.760552,79.839303,1483119699,2016-12-30 17:41:39,NaN,91
5289,"{'timestampMs': '1484396822311', 'latitudeE7':...",10.760552,79.839303,1484396822,2017-01-14 12:27:02,NaN,91


In [21]:
df_gps_sample[df_gps_sample["cluster"]==336].tail(1)

,locations,lat,lon,timestamp_s,datetime,altitude,cluster
45106,"{'timestampMs': '1498981837471', 'latitudeE7':...",10.718643,79.063361,1498981837,2017-07-02 07:50:37,NaN,336
